In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import *
import nltk
import json
import pandas as pd
import numpy as np
import torch

SEED = 42
nltk.download('punkt')

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
model_path = "../../models"

In [3]:
# model to fine tune
model_checkpoint = "cjvt/t5-sl-small"
# model_checkpoint = "cjvt/t5-sl-large"
# model_checkpoint = "google/mt5-small"
# model_checkpoint = "google/mt5-base"
# model_checkpoint = "google/mt5-large"
name = model_checkpoint.split("/")[-1]

epochs = 25

# this batch size fits on a GPU with 12GB of RAM with the model_checkpoint = "cjvt/t5-sl-small"
batch_size = 16
# this batch size fits on a GPU with 12GB of RAM with the model_checkpoint = "google/mt5-small"
# batch_size = 8

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### Load the data

In [4]:
data = pd.read_csv("../../data/processed/prompt_reply_pairs.csv", sep=";")
data

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
47509,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
47510,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
47511,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
47512,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test


In [5]:
train_data = data[data["split"] == "train"]
val_data = data[data["split"] == "val"]
test_data = data[data["split"] == "test"]
len(train_data), len(val_data), len(test_data)

(37902, 4737, 4875)

In [6]:
train_data = Dataset.from_pandas(train_data[['prompt', 'reply']])
val_data = Dataset.from_pandas(val_data[['prompt', 'reply']])
test_data = Dataset.from_pandas(test_data[['prompt', 'reply']])
train_data

Dataset({
    features: ['prompt', 'reply', '__index_level_0__'],
    num_rows: 37902
})

In [7]:
def convert_to_features(examples):
    prefix_in = "Uporabnik: "
    # prefix_in = ""
    examples["prompt"] = [prefix_in + prompt for prompt in examples["prompt"]]
    # prefix_out = "Asistent: "
    prefix_out = ""
    examples["reply"] = [prefix_out + reply for reply in examples["reply"]]
    
    model_inputs = tokenizer(examples['prompt'], pad_to_max_length=True, max_length=512, truncation=True, return_tensors='pt')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reply'], pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_data = train_data.map(convert_to_features, batched=True, load_from_cache_file=False)
val_data = val_data.map(convert_to_features, batched=True, load_from_cache_file=False)
test_data = test_data.map(convert_to_features, batched=True, load_from_cache_file=False)
train_data

Map:   0%|          | 0/37902 [00:00<?, ? examples/s]c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.w

Dataset({
    features: ['prompt', 'reply', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 37902
})

### Fine Tune

In [9]:
metric = load_metric("rouge")

C:\Users\Luka\AppData\Local\Temp\ipykernel_8860\3314070117.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_path}/{name}-finetuned-assistant",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=False, # setting this to true gives loss 0.0 at every step for some reason
    push_to_hub=False, 
    load_best_model_at_end=True # load best model at end so we save the best model instead of the last model
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()
trainer.save_model(f"{model_path}/{name}-finetuned-assistant")

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 500/59225 [01:53<3:37:54,  4.49it/s]

{'loss': 9.3774, 'learning_rate': 1.9831152384972564e-05, 'epoch': 0.21}


  2%|▏         | 1000/59225 [03:45<3:36:32,  4.48it/s]

{'loss': 3.3558, 'learning_rate': 1.9662304769945127e-05, 'epoch': 0.42}


  3%|▎         | 1500/59225 [05:37<3:34:22,  4.49it/s]

...

{'loss': 2.5903, 'learning_rate': 7.598142676234698e-08, 'epoch': 24.91}


                                                       
100%|██████████| 59225/59225 [4:15:56<00:00,  4.77it/s]

{'eval_loss': 2.37141489982605, 'eval_rouge1': 12.4506, 'eval_rouge2': 3.6566, 'eval_rougeL': 10.2485, 'eval_rougeLsum': 11.3412, 'eval_gen_len': 18.7218, 'eval_runtime': 98.3333, 'eval_samples_per_second': 48.173, 'eval_steps_per_second': 3.02, 'epoch': 25.0}


100%|██████████| 59225/59225 [4:15:57<00:00,  3.86it/s]


{'train_runtime': 15357.9704, 'train_samples_per_second': 61.698, 'train_steps_per_second': 3.856, 'train_loss': 2.796831964675562, 'epoch': 25.0}


### Evaluate

Validation set

In [13]:
val_results = trainer.evaluate()
print('Val results: ', val_results)

100%|██████████| 297/297 [01:37<00:00,  3.05it/s]

Val results:  {'eval_loss': 2.37141489982605, 'eval_rouge1': 12.4506, 'eval_rouge2': 3.6566, 'eval_rougeL': 10.2485, 'eval_rougeLsum': 11.3412, 'eval_gen_len': 18.7218, 'eval_runtime': 97.9626, 'eval_samples_per_second': 48.355, 'eval_steps_per_second': 3.032, 'epoch': 25.0}


Test set

In [14]:
test_results = trainer.predict(test_dataset=test_data)
print('Test results:', test_results.metrics)

100%|██████████| 305/305 [01:40<00:00,  3.03it/s]

Test results: {'test_loss': 2.4149272441864014, 'test_rouge1': 12.288, 'test_rouge2': 3.5705, 'test_rougeL': 10.0529, 'test_rougeLsum': 11.2572, 'test_gen_len': 18.6868, 'test_runtime': 100.9741, 'test_samples_per_second': 48.28, 'test_steps_per_second': 3.021}


In [15]:
input = "Uporabnik: Kdo je France Prešeren?"
input = tokenizer(input, return_tensors="pt").to("cuda")
outputs = trainer.model.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu v različnih delih sveta.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju glasbe, glasbe in glasbe.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju umetnosti in umetnosti.',
 'France Prešeren je bil francoski pesnik, ki se je rodil leta 1922 v Parizu. Bil je francoski pisatelj in pisatelj, znan po svojem delu in delu na področju glasbe, glasbe in glasbe. Znan je tudi po svojih delih, kot so pesmi, eseji, pesmi in pesmi.']